In [3]:
import kivy

In [ ]:
import kivy 
kivy.require('1.10.0') 
   
from kivy.app import App 
from kivy.uix.button import Label 

from kivy.core.window import Window
Window.fullscreen = True

# Inherit Kivy's App class which represents the window 
# for our widgets 
# HelloKivy inherits all the fields and methods 
# from Kivy 
class HelloKivy(App): 
   
    # This returns the content we want in the window 
    def build(self): 
   
        # Return a label widget with Hello Kivy 
        return Label(text ="Hello Geeks")

helloKivy = HelloKivy() 
helloKivy.run()

In [2]:
# main.py 
# import the kivy module 
import kivy  
  
# It’s required that the base Class  
# of your App inherits from the App class. 
from kivy.app import App 
from kivy.uix.gridlayout import GridLayout 
  
# This class stores the info of .kv file 
# when it is called goes to my.kv file 
class MainWidget(GridLayout):  
    pass
  
# we are defining the Base Class of our Kivy App 
class myApp(App): 
    def build(self): 
        # return a MainWidget() as a root widget 
        return MainWidget() 
    
# Here the class MyApp is initialized 
# and its run() method called. 
myApp().run() 

In [ ]:
import kivy
from kivy.app import App
from kivy.uix.widget import Widget
from kivy.properties import ObjectProperty
from kivy.uix.label import Label
from kivy.uix.floatlayout import FloatLayout
from kivy.uix.popup import Popup

from kivy.core.window import Window
Window.fullscreen = False


class Widgets(Widget):
    def btn(self):
        show_popup()

class P(FloatLayout):
    pass


class MyApp(App):
    def build(self):
        return Widgets()


def show_popup():
    show = P()

    popupWindow = Popup(title="Popup Window", content=show, size_hint=(None,None),size=(400,400))

    popupWindow.open()


if __name__ == "__main__":
    MyApp().run()

[INFO   ] [Logger      ] Record log in C:\Users\Dandar\.kivy\logs\kivy_20-03-16_7.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.gstreamer" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.2.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.2.0
[INFO   ] [Kivy        ] v1.11.1
[INFO   ] [Kivy        ] Installed at "C:\Users\Dandar\.conda\envs\danzeff-remake\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "C:\Users\Dandar\.conda\envs\danzeff-remake\python.exe"
[INFO   ] [Factory     ] 184 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_gif (img_pil, img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics syste